In [1]:
# Objectives:

# FOR RELEASE 1 MVP
# - Only 2019 data
# - Rename each 2019 data column (population, rent, crime)
# - Create column with city, state (abbreviation?)
# - Column with state
# - Make table for each csv, or just join with pandas
# - Join 3 df/tables on city, state (abbreviation?) column
# - Check for duplicate cities and drop (already drop with inner join)
# - Drop any cities that lack pop, rent, crime data (already drop with inner join, but still drop NaN values and fill later)
# - Bin population, rent, and crime data
# - Categorize population, rent, and crime data

# MACHINE LEARNNG (in upcoming notebook)
# - Vectorize data
# - Train nearest neighbors model on city/state, pop, rent, and crime data
# - Make model into a function
# - Use function to make a recommendation of Location based on population, rent, crime rate
# - Check to see if recommendation matches well with data. If so:
# - Pickle the model and it is ready to be put into API and tested with Web/iOS
# - Once these steps are completed and working, we will also incorporate walkability and livability score in Release 2.
# - When walkability and livability scores are also included and working well in the model, we welcome and further additions to the model, granted the data is from 2019 (otherwise we can include a disclaimer, or we push all of the data used back to 2018, for example, as long as the data all comes from the same year)

# STRETCH GOALS
# - add more data that fit team's user stories
# - attempt forecasting using data from 2010-2020
# - try fb prophet model among other time series models and techniques
# - be in conversation with engineers

In [2]:
# Imports

import pandas as pd
import numpy as np

In [3]:
# Load an inspect population data

population = pd.read_csv("../data/population.csv")
population

,Location,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019 Population
0,"Abbeville city, Alabama","2,688","2,705","2,699","2,694","2,643","2,628","2,608","2,600","2,584","2,575","2,571","2,560"
1,"Adamsville city, Alabama","4,522","4,506","4,500","4,493","4,471","4,449","4,420","4,390","4,356","4,327","4,308","4,281"
2,"Addison town, Alabama",758,754,751,750,743,742,739,734,731,726,723,718
3,"Akron town, Alabama",356,356,355,347,347,343,338,339,333,332,331,328
4,"Alabaster city, Alabama","30,352","31,112","31,209","31,375","31,684","31,980","32,182","32,772","33,017","33,275","33,413","33,487"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19497,"Wamsutter town, Wyoming",451,451,450,453,462,487,508,499,493,483,474,467
19498,"Wheatland town, Wyoming","3,627","3,625","3,622","3,620","3,626","3,622","3,642","3,645","3,587","3,549","3,527","3,462"
19499,"Worland city, Wyoming","5,487","5,487","5,487","5,436","5,419","5,419","5,328","5,332","5,263","5,158","5,071","5,024"
19500,"Wright town, Wyoming","1,807","1,807","1,810","1,812","1,864","1,860","1,856","1,885","1,857","1,760","1,754","1,753"


In [4]:
# Check population data types

population.dtypes

Location           object
Census             object
Estimates Base     object
2010               object
2011               object
2012               object
2013               object
2014               object
2015               object
2016               object
2017               object
2018               object
2019 Population    object
dtype: object

In [5]:
# Drop columns from population data (or can create a copy with just location and 2019 population data)

population = population.drop(['Census', 'Estimates Base', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'], axis=1)
population

# In Location column, after the name of the town or city it contains "town" or "city". Wrangle this out.

,Location,2019 Population
0,"Abbeville city, Alabama","2,560"
1,"Adamsville city, Alabama","4,281"
2,"Addison town, Alabama",718
3,"Akron town, Alabama",328
4,"Alabaster city, Alabama","33,487"
...,...,...
19497,"Wamsutter town, Wyoming",467
19498,"Wheatland town, Wyoming","3,462"
19499,"Worland city, Wyoming","5,024"
19500,"Wright town, Wyoming","1,753"


In [6]:
# Create new column to specify if Location is city or town

population["Town or City"] = population['Location'].str.extract("(city|town)")
population

,Location,2019 Population,Town or City
0,"Abbeville city, Alabama","2,560",city
1,"Adamsville city, Alabama","4,281",city
2,"Addison town, Alabama",718,town
3,"Akron town, Alabama",328,town
4,"Alabaster city, Alabama","33,487",city
...,...,...,...
19497,"Wamsutter town, Wyoming",467,town
19498,"Wheatland town, Wyoming","3,462",town
19499,"Worland city, Wyoming","5,024",city
19500,"Wright town, Wyoming","1,753",town


In [7]:
# Remove city and town strings from Location column so that the column can be used to join with other data

population["Location"] = population['Location'].str.replace(' city', '', regex=False)
population["Location"] = population['Location'].str.replace(' town', '', regex=False)

population

# Consider adding just a State column, having City and State columns separate

,Location,2019 Population,Town or City
0,"Abbeville, Alabama","2,560",city
1,"Adamsville, Alabama","4,281",city
2,"Addison, Alabama",718,town
3,"Akron, Alabama",328,town
4,"Alabaster, Alabama","33,487",city
...,...,...,...
19497,"Wamsutter, Wyoming",467,town
19498,"Wheatland, Wyoming","3,462",town
19499,"Worland, Wyoming","5,024",city
19500,"Wright, Wyoming","1,753",town


In [8]:
# Unique number of population Locations count

population['Location'].nunique()

19491

In [9]:
# Load and inspect rental rates data

rent = pd.read_csv("../data/rental_rates.csv")
rent

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,...,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11
0,61639,10025,1,"New York, NY",3012.0,3025.0,3037.0,3049.0,3062.0,3074.0,...,3268.0,3234.0,3199.0,3162.0,3125.0,3088.0,3048.0,3008.0,2968.0,2925.0
1,84654,60657,2,"Chicago, IL",1588.0,1594.0,1599.0,1605.0,1610.0,1615.0,...,1834.0,1829.0,1824.0,1818.0,1813.0,1807.0,1801.0,1795.0,1788.0,1781.0
2,61637,10023,3,"New York, NY",3114.0,3123.0,3131.0,3140.0,3148.0,3156.0,...,3307.0,3275.0,3244.0,3211.0,3178.0,3144.0,3108.0,3072.0,3035.0,2997.0
3,91982,77494,4,"Houston, TX",1759.0,1763.0,1766.0,1770.0,1773.0,1776.0,...,1775.0,1777.0,1780.0,1782.0,1785.0,1788.0,1791.0,1794.0,1796.0,1799.0
4,84616,60614,5,"Chicago, IL",1740.0,1745.0,1750.0,1755.0,1759.0,1764.0,...,2023.0,2017.0,2010.0,2003.0,1995.0,1988.0,1979.0,1970.0,1962.0,1952.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,62321,11976,9253,"New York, NY",63788.0,NaN,NaN,NaN,NaN,NaN,...,49083.0,48860.0,48636.0,NaN,NaN,NaN,NaN,NaN,NaN,47911.0
3186,58624,2110,9469,"Boston, MA",4113.0,4105.0,4097.0,4089.0,NaN,4077.0,...,4510.0,NaN,NaN,4350.0,4294.0,4239.0,4180.0,4121.0,4062.0,4002.0
3187,66128,20004,9592,"Washington, DC",NaN,NaN,2295.0,2304.0,NaN,2323.0,...,2496.0,2494.0,2492.0,2490.0,2487.0,2484.0,2481.0,2478.0,2475.0,2471.0
3188,399647,80951,9634,"Colorado Springs, CO",NaN,1252.0,1256.0,1260.0,1265.0,1269.0,...,NaN,1656.0,1662.0,1669.0,1676.0,1683.0,1690.0,1697.0,1704.0,1712.0


In [10]:
# Check rent data types

rent.dtypes

RegionID        int64
RegionName      int64
SizeRank        int64
MsaName        object
2014-01       float64
               ...   
2020-07       float64
2020-08       float64
2020-09       float64
2020-10       float64
2020-11       float64
Length: 87, dtype: object

In [11]:
# Drop all columns you dont want, or just use code below to make new df

#rent = rent.drop(['RegionID', 'RegionName', 'SizeRank', '2014-01', '2014-02', '2014-03', '2014-04', ...], axis=1)

In [12]:
# Create copy of rental rates dataframe and change column names

rent = rent[['MsaName','2019-12']].copy()
rent = rent.rename(columns = {"MsaName":"Location"})
rent = rent.rename(columns = {"2019-12":"2019 Rental Rates"})
rent

,Location,2019 Rental Rates
0,"New York, NY",3311.0
1,"Chicago, IL",1838.0
2,"New York, NY",3344.0
3,"Houston, TX",1772.0
4,"Chicago, IL",2028.0
...,...,...
3185,"New York, NY",50175.0
3186,"Boston, MA",4585.0
3187,"Washington, DC",2495.0
3188,"Colorado Springs, CO",1640.0


In [13]:
# Replace state abbreviations with full state names

rent["Location"] = rent['Location'].str.replace('AK', 'Alaska', regex=False)
rent["Location"] = rent['Location'].str.replace('AL', 'Alabama', regex=False)
rent["Location"] = rent['Location'].str.replace('AR', 'Arkansas', regex=False)
rent["Location"] = rent['Location'].str.replace('AS', 'American Samoa', regex=False)
rent["Location"] = rent['Location'].str.replace('AZ', 'Arizona', regex=False)
rent["Location"] = rent['Location'].str.replace('CA', 'California', regex=False)
rent["Location"] = rent['Location'].str.replace('CO', 'Colorado', regex=False)
rent["Location"] = rent['Location'].str.replace('CT', 'Connecticut', regex=False)
rent["Location"] = rent['Location'].str.replace('DC', 'District of Columbia', regex=False)
rent["Location"] = rent['Location'].str.replace('DE', 'Delaware', regex=False)
rent["Location"] = rent['Location'].str.replace('FL', 'Florida', regex=False)
rent["Location"] = rent['Location'].str.replace('GA', 'Georgia', regex=False)
rent["Location"] = rent['Location'].str.replace('GU', 'Guam', regex=False)
rent["Location"] = rent['Location'].str.replace('HI', 'Hawaii', regex=False)
rent["Location"] = rent['Location'].str.replace('IA', 'Iowa', regex=False)
rent["Location"] = rent['Location'].str.replace('ID', 'Idaho', regex=False)
rent["Location"] = rent['Location'].str.replace('IL', 'Illinois', regex=False)
rent["Location"] = rent['Location'].str.replace('IN', 'Indiana', regex=False)
rent["Location"] = rent['Location'].str.replace('KS', 'Kansas', regex=False)
rent["Location"] = rent['Location'].str.replace('KY', 'Kentucky', regex=False)
rent["Location"] = rent['Location'].str.replace('LA', 'Louisiana', regex=False)
rent["Location"] = rent['Location'].str.replace('MA', 'Massachusetts', regex=False)
rent["Location"] = rent['Location'].str.replace('MD', 'Maryland', regex=False)
rent["Location"] = rent['Location'].str.replace('ME', 'Maine', regex=False)
rent["Location"] = rent['Location'].str.replace('MI', 'Michigan', regex=False)
rent["Location"] = rent['Location'].str.replace('MN', 'Minnesota', regex=False)
rent["Location"] = rent['Location'].str.replace('MO', 'Missouri', regex=False)
rent["Location"] = rent['Location'].str.replace('MP', 'Northern Mariana Islands', regex=False)
rent["Location"] = rent['Location'].str.replace('MS', 'Mississippi', regex=False)
rent["Location"] = rent['Location'].str.replace('MT', 'Montana', regex=False)
rent["Location"] = rent['Location'].str.replace('NA', 'National', regex=False)
rent["Location"] = rent['Location'].str.replace('NC', 'North Carolina', regex=False)
rent["Location"] = rent['Location'].str.replace('ND', 'North Dakota', regex=False)
rent["Location"] = rent['Location'].str.replace('NE', 'Nebraska', regex=False)
rent["Location"] = rent['Location'].str.replace('NH', 'New Hampshire', regex=False)
rent["Location"] = rent['Location'].str.replace('NJ', 'New Jersey', regex=False)
rent["Location"] = rent['Location'].str.replace('NM', 'New Mexico', regex=False)
rent["Location"] = rent['Location'].str.replace('NV', 'Nevada', regex=False)
rent["Location"] = rent['Location'].str.replace('NY', 'New York', regex=False)
rent["Location"] = rent['Location'].str.replace('OH', 'Ohio', regex=False)
rent["Location"] = rent['Location'].str.replace('OK', 'Oklahoma', regex=False)
rent["Location"] = rent['Location'].str.replace('OR', 'Oregon', regex=False)
rent["Location"] = rent['Location'].str.replace('PA', 'Pennsylvania', regex=False)
rent["Location"] = rent['Location'].str.replace('PR', 'Puerto Rico', regex=False)
rent["Location"] = rent['Location'].str.replace('RI', 'Rhode Island', regex=False)
rent["Location"] = rent['Location'].str.replace('SC', 'South Carolina', regex=False)
rent["Location"] = rent['Location'].str.replace('SD', 'South Dakota', regex=False)
rent["Location"] = rent['Location'].str.replace('TN', 'Tennessee', regex=False)
rent["Location"] = rent['Location'].str.replace('TX', 'Texas', regex=False)
rent["Location"] = rent['Location'].str.replace('UT', 'Utah', regex=False)
rent["Location"] = rent['Location'].str.replace('VA', 'Virginia', regex=False)
rent["Location"] = rent['Location'].str.replace('VI', 'Virgin Islands', regex=False)
rent["Location"] = rent['Location'].str.replace('VT', 'Vermont', regex=False)
rent["Location"] = rent['Location'].str.replace('WA', 'Washington', regex=False)
rent["Location"] = rent['Location'].str.replace('WI', 'Wisconsin', regex=False)
rent["Location"] = rent['Location'].str.replace('WV', 'West Virginia', regex=False)
rent["Location"] = rent['Location'].str.replace('WY', 'Wyoming', regex=False)
rent

,Location,2019 Rental Rates
0,"New York, New York",3311.0
1,"Chicago, Illinois",1838.0
2,"New York, New York",3344.0
3,"Houston, Texas",1772.0
4,"Chicago, Illinois",2028.0
...,...,...
3185,"New York, New York",50175.0
3186,"Boston, Massachusetts",4585.0
3187,"Washington, District of Columbia",2495.0
3188,"Colorado Springs, Colorado",1640.0


In [14]:
# Unique number of rent Locations count

rent['Location'].nunique()

101

In [15]:
# Load and inspect crime rates data (need to replicate state for every city)

crime = pd.read_csv("../data/crime_rates.csv")
crime

,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,Alabama,Hoover,"85,670",114,4.0,15,27,68,"1,922",128,"1,694",100,2
1,Alaska,Anchorage,"287,731","3,581",32.0,540,621,"2,388","12,261","1,692","9,038","1,531",93
2,Alaska,Bethel,"6,544",130,1.0,47,3,79,132,20,84,28,12
3,Alaska,Bristol Bay Borough,852,2,0.0,0,0,2,20,5,8,7,0
4,Alaska,Cordova,"2,150",0,0.0,0,0,0,7,1,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,Wyoming,Sheridan,"17,895",9,0.0,4,0,5,369,75,278,16,3
8101,Wyoming,Thermopolis,"2,830",13,0.0,0,0,13,34,7,22,5,0
8102,Wyoming,Torrington,"6,709",13,0.0,4,1,8,48,8,40,0,0
8103,Wyoming,Wheatland,"3,544",7,0.0,1,0,6,72,24,45,3,0


In [16]:
# Check crime data types

crime.dtypes

State                                    object
City                                     object
Population                               object
Violent crime                            object
Murder and nonnegligent manslaughter    float64
Rape                                     object
Robbery                                  object
Aggravated assault                       object
Property crime                           object
Burglary                                 object
Larceny-theft                            object
Motor vehicle theft                      object
Arson                                    object
dtype: object

In [17]:
# Make new Location column with City and State data, so the data can be joineed with the other data on this column

crime['Location'] = crime['City'] + ',' + ' ' + crime['State']
crime

# Consider adding Alabama crime data from 2018

,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Location
0,Alabama,Hoover,"85,670",114,4.0,15,27,68,"1,922",128,"1,694",100,2,"Hoover, Alabama"
1,Alaska,Anchorage,"287,731","3,581",32.0,540,621,"2,388","12,261","1,692","9,038","1,531",93,"Anchorage, Alaska"
2,Alaska,Bethel,"6,544",130,1.0,47,3,79,132,20,84,28,12,"Bethel, Alaska"
3,Alaska,Bristol Bay Borough,852,2,0.0,0,0,2,20,5,8,7,0,"Bristol Bay Borough, Alaska"
4,Alaska,Cordova,"2,150",0,0.0,0,0,0,7,1,6,0,0,"Cordova, Alaska"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,Wyoming,Sheridan,"17,895",9,0.0,4,0,5,369,75,278,16,3,"Sheridan, Wyoming"
8101,Wyoming,Thermopolis,"2,830",13,0.0,0,0,13,34,7,22,5,0,"Thermopolis, Wyoming"
8102,Wyoming,Torrington,"6,709",13,0.0,4,1,8,48,8,40,0,0,"Torrington, Wyoming"
8103,Wyoming,Wheatland,"3,544",7,0.0,1,0,6,72,24,45,3,0,"Wheatland, Wyoming"


In [18]:
# Unique number of crime Locations count

crime['Location'].nunique()

# 19491 unique Locations in population dataset
# 101 unique Locations in rent dataset   -   (Consider looking for a different rent data set with more unique locations)
# 8105 unique Locations in crime dataset

8105

In [19]:
# When wrangling is done here, combine tables, bin data in new columns, and push data to PG DB. 
 
# Example of Joins

merged_population_rent = pd.merge(left=population, right=rent, left_on='Location', right_on='Location')
merged_population_rent

,Location,2019 Population,Town or City,2019 Rental Rates
0,"Birmingham, Alabama","209,403",city,935.0
1,"Birmingham, Alabama","209,403",city,1328.0
2,"Birmingham, Alabama","209,403",city,1675.0
3,"Birmingham, Alabama","209,403",city,966.0
4,"Birmingham, Alabama","209,403",city,1426.0
...,...,...,...,...
2502,"Milwaukee, Wisconsin","590,157",city,839.0
2503,"Milwaukee, Wisconsin","590,157",city,948.0
2504,"Milwaukee, Wisconsin","590,157",city,973.0
2505,"Milwaukee, Wisconsin","590,157",city,792.0


In [20]:
# Unique number of merged_population_rent Locations count

merged_population_rent['Location'].nunique()

# There are 11 unique Locations being lost from the dataset because the population and rent data sets do not have all of the same Locations

90

In [21]:
# Merge population, rent, and crime data which can be pushed to DB. Data can also be added to this, more data preserved, and binned.

merged_population_rent_crime = pd.merge(left=merged_population_rent, right=crime, left_on='Location', right_on='Location')
merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson
0,"Phoenix, Arizona","1,680,992",city,1424.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
1,"Phoenix, Arizona","1,680,992",city,1522.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
2,"Phoenix, Arizona","1,680,992",city,1461.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
3,"Phoenix, Arizona","1,680,992",city,1549.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
4,"Phoenix, Arizona","1,680,992",city,1611.0,Arizona,Phoenix,"1,688,722","11,803",131.0,"1,139","3,197","7,336","55,974","9,471","39,427","7,076",201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin","590,157",city,839.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198
2046,"Milwaukee, Wisconsin","590,157",city,948.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198
2047,"Milwaukee, Wisconsin","590,157",city,973.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198
2048,"Milwaukee, Wisconsin","590,157",city,792.0,Wisconsin,Milwaukee,"590,923","7,874",97.0,427,"1,911","5,439","15,097","3,594","8,053","3,450",198


In [22]:
# Unique number of merged_population_rent_crime Locations count

merged_population_rent_crime['Location'].nunique()

# There are 14 more unique Locations being lost from the dataset because the population and rent data sets do not have all of the same Locations

76

In [23]:
# Check data types before binning

merged_population_rent_crime.dtypes

Location                                 object
2019 Population                          object
Town or City                             object
2019 Rental Rates                       float64
State                                    object
City                                     object
Population                               object
Violent crime                            object
Murder and nonnegligent manslaughter    float64
Rape                                     object
Robbery                                  object
Aggravated assault                       object
Property crime                           object
Burglary                                 object
Larceny-theft                            object
Motor vehicle theft                      object
Arson                                    object
dtype: object

In [24]:
# Remove commas from columns and change data type to floats

merged_population_rent_crime['2019 Population'] = merged_population_rent_crime['2019 Population'].replace(',','', regex=True)
merged_population_rent_crime['Crime Population'] = merged_population_rent_crime['Population'].replace(',','', regex=True)
merged_population_rent_crime = merged_population_rent_crime.drop(['Population'], axis=1)
merged_population_rent_crime['Violent crime'] = merged_population_rent_crime['Violent crime'].replace(',','', regex=True)
merged_population_rent_crime['Murder and nonnegligent manslaughter'] = merged_population_rent_crime['Murder and nonnegligent manslaughter'].replace(',','', regex=True)
merged_population_rent_crime['Rape'] = merged_population_rent_crime['Rape'].replace(',','', regex=True)
merged_population_rent_crime['Robbery'] = merged_population_rent_crime['Robbery'].replace(',','', regex=True)
merged_population_rent_crime['Aggravated assault'] = merged_population_rent_crime['Aggravated assault'].replace(',','', regex=True)
merged_population_rent_crime['Property crime'] = merged_population_rent_crime['Property crime'].replace(',','', regex=True)
merged_population_rent_crime['Burglary'] = merged_population_rent_crime['Burglary'].replace(',','', regex=True)
merged_population_rent_crime['Larceny-theft'] = merged_population_rent_crime['Larceny-theft'].replace(',','', regex=True)
merged_population_rent_crime['Motor vehicle theft'] = merged_population_rent_crime['Motor vehicle theft'].replace(',','', regex=True)
merged_population_rent_crime['Arson'] = merged_population_rent_crime['Arson'].replace(',','', regex=True)

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Population
0,"Phoenix, Arizona",1680992,city,1424.0,Arizona,Phoenix,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201,1688722
1,"Phoenix, Arizona",1680992,city,1522.0,Arizona,Phoenix,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201,1688722
2,"Phoenix, Arizona",1680992,city,1461.0,Arizona,Phoenix,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201,1688722
3,"Phoenix, Arizona",1680992,city,1549.0,Arizona,Phoenix,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201,1688722
4,"Phoenix, Arizona",1680992,city,1611.0,Arizona,Phoenix,11803,131.0,1139,3197,7336,55974,9471,39427,7076,201,1688722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839.0,Wisconsin,Milwaukee,7874,97.0,427,1911,5439,15097,3594,8053,3450,198,590923
2046,"Milwaukee, Wisconsin",590157,city,948.0,Wisconsin,Milwaukee,7874,97.0,427,1911,5439,15097,3594,8053,3450,198,590923
2047,"Milwaukee, Wisconsin",590157,city,973.0,Wisconsin,Milwaukee,7874,97.0,427,1911,5439,15097,3594,8053,3450,198,590923
2048,"Milwaukee, Wisconsin",590157,city,792.0,Wisconsin,Milwaukee,7874,97.0,427,1911,5439,15097,3594,8053,3450,198,590923


In [25]:
# Check for null values

merged_population_rent_crime.isnull().sum().sum()

440

In [26]:
# Drop all null values for now, can return to preserve rows by filling (fillna(), replace() and interpolate())

merged_population_rent_crime = merged_population_rent_crime.dropna()
merged_population_rent_crime.shape

(1626, 17)

In [27]:
# Change datatype to float or int, int looks better with no decimal and trailing 0

#merged_population_rent_crime['2019 Population'].astype('float64')
#merged_population_rent_crime['2019 Rental Rates'].astype('float64')
#merged_population_rent_crime['Population'].astype('float64')
#merged_population_rent_crime['Violent crime'].astype('float64')
#merged_population_rent_crime['Murder and nonnegligent manslaughter'].astype('float64')
#merged_population_rent_crime['Rape'].astype('float64')
#merged_population_rent_crime['Robbery'].astype('float64')
#merged_population_rent_crime['Aggravated assault'].astype('float64')
#merged_population_rent_crime['Property crime'].astype('float64')
#merged_population_rent_crime['Burglary'].astype('float64')
#merged_population_rent_crime['Larceny-theft'].astype('float64')
#merged_population_rent_crime['Motor vehicle theft'].astype('float64')
#merged_population_rent_crime['Arson'].astype('float64')

merged_population_rent_crime['2019 Population'] = merged_population_rent_crime['2019 Population'].astype('int')
merged_population_rent_crime['2019 Rental Rates'] = merged_population_rent_crime['2019 Rental Rates'].astype('int') # ValueError: Cannot convert non-finite values (NA or inf) to integer
#merged_population_rent_crime['Population'] = merged_population_rent_crime['Population'].astype('int')
merged_population_rent_crime['Violent crime'] = merged_population_rent_crime['Violent crime'].astype('int')
merged_population_rent_crime['Murder and nonnegligent manslaughter'] = merged_population_rent_crime['Murder and nonnegligent manslaughter'].astype('int')
merged_population_rent_crime['Rape'] = merged_population_rent_crime['Rape'].astype('int')
merged_population_rent_crime['Robbery'] = merged_population_rent_crime['Robbery'].astype('int')
merged_population_rent_crime['Aggravated assault'] = merged_population_rent_crime['Aggravated assault'].astype('int')
merged_population_rent_crime['Property crime'] = merged_population_rent_crime['Property crime'].astype('int') # ValueError: cannot convert float NaN to integer
merged_population_rent_crime['Burglary'] = merged_population_rent_crime['Burglary'].astype('int') # ValueError: cannot convert float NaN to integer
merged_population_rent_crime['Larceny-theft'] = merged_population_rent_crime['Larceny-theft'].astype('int')
merged_population_rent_crime['Motor vehicle theft'] = merged_population_rent_crime['Motor vehicle theft'].astype('int')
merged_population_rent_crime['Arson'] = merged_population_rent_crime['Arson'].astype('int') # ValueError: cannot convert float NaN to integer

# These columns previously had NA or NaN values which had to be dropped. These columns can be revisited to preserve more data.

#merged_population_rent_crime['2019 Rental Rates']
#merged_population_rent_crime['Property crime']
#merged_population_rent_crime['Burglary']
#merged_population_rent_crime['Arson']

In [28]:
# Check data types again before binning, since changing

merged_population_rent_crime.dtypes

Location                                object
2019 Population                          int32
Town or City                            object
2019 Rental Rates                        int32
State                                   object
City                                    object
Violent crime                            int32
Murder and nonnegligent manslaughter     int32
Rape                                     int32
Robbery                                  int32
Aggravated assault                       int32
Property crime                           int32
Burglary                                 int32
Larceny-theft                            int32
Motor vehicle theft                      int32
Arson                                    int32
Crime Population                        object
dtype: object

In [29]:
# Combine crime data into new column and divide by population * 1000 (This is how crime rate is calculated)

merged_population_rent_crime['Crime Rate']=merged_population_rent_crime.iloc[:,-11:].sum(axis=1)/merged_population_rent_crime['2019 Population']*100000

#merged_population_rent_crime['Crime Rate'] = merged_population_rent_crime['Violent crime'] + merged_population_rent_crime['Murder and nonnegligent manslaughter'] + #merged_population_rent_crime['Rape'] + merged_population_rent_crime + merged_population_rent_crime['Robbery'] + ['Aggravated assault'] + merged_population_rent_crime['Property #crime'] + merged_population_rent_crime['Burglary'] + merged_population_rent_crime['Larceny-theft'] + merged_population_rent_crime['Motor vehicle theft'] + #merged_population_rent_crime['Arson']/merged_population_rent_crime['Population']*100000

#merged_population_rent_crime['Crime Rate'] = total_crimes/merged_population_rent_crime['Population']*100000

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Population,Crime Rate
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531


In [30]:
# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime['2019 Population'].describe()

count    1.626000e+03
mean     8.890544e+05
std      7.568980e+05
min      4.927100e+04
25%      3.313600e+05
50%      6.547410e+05
75%      1.021795e+06
max      2.693976e+06
Name: 2019 Population, dtype: float64

In [31]:
# Quartile cut the data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime['2019 Population'], q=6)

0       (881549.0, 1680992.0]
1       (881549.0, 1680992.0]
2       (881549.0, 1680992.0]
3       (881549.0, 1680992.0]
4       (881549.0, 1680992.0]
                ...          
2045     (399700.0, 654741.0]
2046     (399700.0, 654741.0]
2047     (399700.0, 654741.0]
2048     (399700.0, 654741.0]
2049     (399700.0, 654741.0]
Name: 2019 Population, Length: 1626, dtype: category
Categories (6, interval[float64]): [(49270.999, 259680.0] < (259680.0, 399700.0] < (399700.0, 654741.0] < (654741.0, 881549.0] < (881549.0, 1680992.0] < (1680992.0, 2693976.0]]

In [32]:
# Bin 2019 population into Urban population by City Size Categories and ranges

# (https://data.oecd.org/popregion/urban-population-by-city-size.htm#:~:text=their%20administrative%20boundaries.-,Urban%20areas%20in%20OECD%20countries%20are%20classified%20as%3A%20large%20metropolitan,areas%20if%20their%20population%20is)

# Urban areas in OECD countries are classified as: large metropolitan areas if they have a population of 1.5 million or more; metropolitan areas if their population is between 500 000 and 1.5 million; medium-size urban areas if their population is between 200 000 and 500 000; and, small urban areas if their population is between 50 000 and 200 000. This indicator is measured as a percentage of the national population.

# Qcut

#merged_population_rent_crime['Urban Population by City Size'] = pd.qcut(merged_population_rent_crime['2019 Population'], q=4, labels = ["Small Urban Area", "Medium-size Urban Area", "Metropolitan Area", "Large Metropolitan Area"])

#merged_population_rent_crime['Population by City Size'] = pd.qcut(merged_population_rent_crime['2019 Population'], q=4)

#merged_population_rent_crime

# Consider using just cut to fit the labels above. qcut will make the bins equal, but it is difficult to categorize cities based on those numbers.

bins = [0, 50000, 200000, 500000, 1500000, 100000000]
labels = ["Town","Small Urban Area", "Medium-size Urban Area", "Metropolitan Area", "Large Metropolitan Area"]
merged_population_rent_crime['Urban Population by City Size Categories'] = pd.cut(merged_population_rent_crime['2019 Population'], bins=bins, labels=labels)

bins = [0, 50000, 200000, 500000, 1500000, 100000000]
labels = ["0-50,000", "50,000-200,000", "200,000-500,000", "500,000-1,500,000", "1,500,000 <"]
merged_population_rent_crime['Urban Population by City Size Ranges'] = pd.cut(merged_population_rent_crime['2019 Population'], bins=bins, labels=labels)


merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Population,Crime Rate,Urban Population by City Size Categories,Urban Population by City Size Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,7336,55974,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,5439,15097,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000"


In [33]:
# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime['2019 Rental Rates'].describe()

count    1626.000000
mean     1674.565806
std       607.902951
min       582.000000
25%      1294.500000
50%      1549.000000
75%      1891.500000
max      6373.000000
Name: 2019 Rental Rates, dtype: float64

In [34]:
# Quartile cut the 2019 Rental Rates data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime['2019 Rental Rates'], q=4)

0        (1294.5, 1549.0]
1        (1294.5, 1549.0]
2        (1294.5, 1549.0]
3        (1294.5, 1549.0]
4        (1549.0, 1891.5]
              ...        
2045    (581.999, 1294.5]
2046    (581.999, 1294.5]
2047    (581.999, 1294.5]
2048    (581.999, 1294.5]
2049    (581.999, 1294.5]
Name: 2019 Rental Rates, Length: 1626, dtype: category
Categories (4, interval[float64]): [(581.999, 1294.5] < (1294.5, 1549.0] < (1549.0, 1891.5] < (1891.5, 6373.0]]

In [35]:
# Bin the 2019 Rental Rates data into categories and ranges

bins = [0, 581.99, 1294.5, 1549, 1891.5, 6373]
labels = ["Lowest Rent","Low Rent", "Average Rent", "Higher Rent", "Highest Rent"]
merged_population_rent_crime['Rental Rate Categories'] = pd.cut(merged_population_rent_crime['2019 Rental Rates'], bins=bins, labels=labels)

bins = [0, 581.99, 1294.5, 1549, 1891.5, 6373]
labels = ["$0-581.99","$581.99-1,294.50", "$1,294.50-1,549", "$1,549-1,891.50", "$1891.50-6,373"]
merged_population_rent_crime['Rental Rate Ranges'] = pd.cut(merged_population_rent_crime['2019 Rental Rates'], bins=bins, labels=labels)

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,...,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Population,Crime Rate,Urban Population by City Size Categories,Urban Population by City Size Ranges,Rental Rate Categories,Rental Rate Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,...,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,...,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,...,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,...,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,...,9471,39427,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Higher Rent,"$1,549-1,891.50"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,...,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,...,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,...,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,...,3594,8053,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50"


In [36]:
# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime['Crime Rate'].describe()

count     1626.000000
mean      8782.427187
std       3034.576864
min       1972.791138
25%       6949.239498
50%       8162.208385
75%      10968.517430
max      16274.419781
Name: Crime Rate, dtype: float64

In [37]:
# Quartile cut the data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime['Crime Rate'], q=5)

0       (7951.131, 9683.737]
1       (7951.131, 9683.737]
2       (7951.131, 9683.737]
3       (7951.131, 9683.737]
4       (7951.131, 9683.737]
                ...         
2045    (5831.013, 7951.131]
2046    (5831.013, 7951.131]
2047    (5831.013, 7951.131]
2048    (5831.013, 7951.131]
2049    (5831.013, 7951.131]
Name: Crime Rate, Length: 1626, dtype: category
Categories (5, interval[float64]): [(1972.79, 5831.013] < (5831.013, 7951.131] < (7951.131, 9683.737] < (9683.737, 11316.718] < (11316.718, 16274.42]]

In [38]:
# Bin the 2019 Crimes Rate data into categories and ranges

bins = [1972.79, 5831.013, 7951.131, 9683.737, 11316.718, 16274.42]
labels = ["Lowest Crime", "Lower Crime", "Average Crime", "Higher Crime", "Highest Crime"]
merged_population_rent_crime['Crime Rate Categories'] = pd.cut(merged_population_rent_crime['Crime Rate'], bins=bins, labels=labels)

bins = [1972.79, 5831.013, 7951.131, 9683.737, 11316.718, 16274.42]
labels = ["1,972.79-5,831.013", "5,831.013-7,951.131", "7,951.131-9,683.737", "9,683.737-11,316.718", "11,316.718-16,274.42"]
merged_population_rent_crime['Crime Rate Ranges'] = pd.cut(merged_population_rent_crime['Crime Rate'], bins=bins, labels=labels)

merged_population_rent_crime

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,...,Motor vehicle theft,Arson,Crime Population,Crime Rate,Urban Population by City Size Categories,Urban Population by City Size Ranges,Rental Rate Categories,Rental Rate Ranges,Crime Rate Categories,Crime Rate Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,...,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,...,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,...,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,...,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,...,7076,201,1688722,8075.886143,Large Metropolitan Area,"1,500,000 <",Higher Rent,"$1,549-1,891.50",Average Crime,"7,951.131-9,683.737"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,...,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131"
2046,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,...,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131"
2047,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,...,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131"
2048,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,...,3450,198,590923,7818.258531,Metropolitan Area,"500,000-1,500,000",Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131"


In [39]:
# Export merged_population_rent_crime with binned categories for csv and beginning modeling in another notebook

merged_population_rent_crime.to_csv('../data/pop_rent_crime_bins.csv')

In [40]:
#1/24/2020 Adding walk_score and cost_of_living data for Release 2

walk = pd.read_csv("../data/walk_score.csv")
cost = pd.read_csv("../data/cost_of_living.csv")

In [41]:
# Check out walk data

walk

,Unnamed: 0,City,Zip Code,Walk Score,Transit Score,Bike Score,Population,State
0,0,Birmingham,35211.0,35,25,31,212237,AL
1,1,Montgomery,36109.0,26,16,38,205764,AL
2,2,Mobile,36605.0,32,--,39,195111,AL
3,3,Huntsville,35810.0,24,13,40,180105,AL
4,4,Tuscaloosa,NaN,33,--,37,90468,AL
...,...,...,...,...,...,...,...,...
2495,1,Casper,NaN,39,--,44,55316,WY
2496,2,Laramie,NaN,34,--,67,30816,WY
2497,3,Gillette,NaN,21,--,34,29087,WY
2498,4,Rock Springs,NaN,29,--,32,23036,WY


In [42]:
# Replace state abbreviations with full state names

walk["State"] = walk['State'].str.replace('AK', 'Alaska', regex=False)
walk["State"] = walk['State'].str.replace('AL', 'Alabama', regex=False)
walk["State"] = walk['State'].str.replace('AR', 'Arkansas', regex=False)
walk["State"] = walk['State'].str.replace('AS', 'American Samoa', regex=False)
walk["State"] = walk['State'].str.replace('AZ', 'Arizona', regex=False)
walk["State"] = walk['State'].str.replace('CA', 'California', regex=False)
walk["State"] = walk['State'].str.replace('CO', 'Colorado', regex=False)
walk["State"] = walk['State'].str.replace('CT', 'Connecticut', regex=False)
walk["State"] = walk['State'].str.replace('DC', 'District of Columbia', regex=False)
walk["State"] = walk['State'].str.replace('DE', 'Delaware', regex=False)
walk["State"] = walk['State'].str.replace('FL', 'Florida', regex=False)
walk["State"] = walk['State'].str.replace('GA', 'Georgia', regex=False)
walk["State"] = walk['State'].str.replace('GU', 'Guam', regex=False)
walk["State"] = walk['State'].str.replace('HI', 'Hawaii', regex=False)
walk["State"] = walk['State'].str.replace('IA', 'Iowa', regex=False)
walk["State"] = walk['State'].str.replace('ID', 'Idaho', regex=False)
walk["State"] = walk['State'].str.replace('IL', 'Illinois', regex=False)
walk["State"] = walk['State'].str.replace('IN', 'Indiana', regex=False)
walk["State"] = walk['State'].str.replace('KS', 'Kansas', regex=False)
walk["State"] = walk['State'].str.replace('KY', 'Kentucky', regex=False)
walk["State"] = walk['State'].str.replace('LA', 'Louisiana', regex=False)
walk["State"] = walk['State'].str.replace('MA', 'Massachusetts', regex=False)
walk["State"] = walk['State'].str.replace('MD', 'Maryland', regex=False)
walk["State"] = walk['State'].str.replace('ME', 'Maine', regex=False)
walk["State"] = walk['State'].str.replace('MI', 'Michigan', regex=False)
walk["State"] = walk['State'].str.replace('MN', 'Minnesota', regex=False)
walk["State"] = walk['State'].str.replace('MO', 'Missouri', regex=False)
walk["State"] = walk['State'].str.replace('MP', 'Northern Mariana Islands', regex=False)
walk["State"] = walk['State'].str.replace('MS', 'Mississippi', regex=False)
walk["State"] = walk['State'].str.replace('MT', 'Montana', regex=False)
walk["State"] = walk['State'].str.replace('NA', 'National', regex=False)
walk["State"] = walk['State'].str.replace('NC', 'North Carolina', regex=False)
walk["State"] = walk['State'].str.replace('ND', 'North Dakota', regex=False)
walk["State"] = walk['State'].str.replace('NE', 'Nebraska', regex=False)
walk["State"] = walk['State'].str.replace('NH', 'New Hampshire', regex=False)
walk["State"] = walk['State'].str.replace('NJ', 'New Jersey', regex=False)
walk["State"] = walk['State'].str.replace('NM', 'New Mexico', regex=False)
walk["State"] = walk['State'].str.replace('NV', 'Nevada', regex=False)
walk["State"] = walk['State'].str.replace('NY', 'New York', regex=False)
walk["State"] = walk['State'].str.replace('OH', 'Ohio', regex=False)
walk["State"] = walk['State'].str.replace('OK', 'Oklahoma', regex=False)
walk["State"] = walk['State'].str.replace('OR', 'Oregon', regex=False)
walk["State"] = walk['State'].str.replace('PA', 'Pennsylvania', regex=False)
walk["State"] = walk['State'].str.replace('PR', 'Puerto Rico', regex=False)
walk["State"] = walk['State'].str.replace('RI', 'Rhode Island', regex=False)
walk["State"] = walk['State'].str.replace('SC', 'South Carolina', regex=False)
walk["State"] = walk['State'].str.replace('SD', 'South Dakota', regex=False)
walk["State"] = walk['State'].str.replace('TN', 'Tennessee', regex=False)
walk["State"] = walk['State'].str.replace('TX', 'Texas', regex=False)
walk["State"] = walk['State'].str.replace('UT', 'Utah', regex=False)
walk["State"] = walk['State'].str.replace('VA', 'Virginia', regex=False)
walk["State"] = walk['State'].str.replace('VI', 'Virgin Islands', regex=False)
walk["State"] = walk['State'].str.replace('VT', 'Vermont', regex=False)
walk["State"] = walk['State'].str.replace('WA', 'Washington', regex=False)
walk["State"] = walk['State'].str.replace('WI', 'Wisconsin', regex=False)
walk["State"] = walk['State'].str.replace('WV', 'West Virginia', regex=False)
walk["State"] = walk['State'].str.replace('WY', 'Wyoming', regex=False)

In [43]:
# Drop the Unnamed: 0 columns, which seems like an extra sort of index

walk = walk.drop(['Unnamed: 0'], axis=1)
walk

,City,Zip Code,Walk Score,Transit Score,Bike Score,Population,State
0,Birmingham,35211.0,35,25,31,212237,Alabama
1,Montgomery,36109.0,26,16,38,205764,Alabama
2,Mobile,36605.0,32,--,39,195111,Alabama
3,Huntsville,35810.0,24,13,40,180105,Alabama
4,Tuscaloosa,NaN,33,--,37,90468,Alabama
...,...,...,...,...,...,...,...
2495,Casper,NaN,39,--,44,55316,Wyoming
2496,Laramie,NaN,34,--,67,30816,Wyoming
2497,Gillette,NaN,21,--,34,29087,Wyoming
2498,Rock Springs,NaN,29,--,32,23036,Wyoming


In [44]:
# Check dtypes and notice Transit Score is an object because values contain "--". Should these values be replaced with 0's?

walk.dtypes

City              object
Zip Code         float64
Walk Score         int64
Transit Score     object
Bike Score         int64
Population         int64
State             object
dtype: object

In [45]:
# Count null values in all of the walk columns

walk.isnull().sum(axis = 0)

City                0
Zip Code         1489
Walk Score          0
Transit Score       0
Bike Score          0
Population          0
State               0
dtype: int64

In [46]:
 # Fill the null values so that we can change Zip Code column to an integer format and maybe use it join data later
 
 walk = walk.fillna(0)

In [47]:
# Change the Zip Code column to be integer type data

walk['Zip Code'] = walk['Zip Code'].astype(int)

In [48]:
# Make new Location column with City and State data, so the data can be joined with the other data on this column

walk['Location'] = walk['City'] + ',' + ' ' + walk['State']
walk

,City,Zip Code,Walk Score,Transit Score,Bike Score,Population,State,Location
0,Birmingham,35211,35,25,31,212237,Alabama,"Birmingham, Alabama"
1,Montgomery,36109,26,16,38,205764,Alabama,"Montgomery, Alabama"
2,Mobile,36605,32,--,39,195111,Alabama,"Mobile, Alabama"
3,Huntsville,35810,24,13,40,180105,Alabama,"Huntsville, Alabama"
4,Tuscaloosa,0,33,--,37,90468,Alabama,"Tuscaloosa, Alabama"
...,...,...,...,...,...,...,...,...
2495,Casper,0,39,--,44,55316,Wyoming,"Casper, Wyoming"
2496,Laramie,0,34,--,67,30816,Wyoming,"Laramie, Wyoming"
2497,Gillette,0,21,--,34,29087,Wyoming,"Gillette, Wyoming"
2498,Rock Springs,0,29,--,32,23036,Wyoming,"Rock Springs, Wyoming"


In [49]:
# Drop City and State columns since they are redundant with Location and they already exist in first merged data

walk = walk.drop(columns=['City', 'State'])

In [50]:
# Bin the walk score according to quartiles(also Bike Score and Transit score but need to investigate and missing value denoted as  "--")

# Describe the data to begin binning data with pandas cut or qcut

walk.describe()

,Zip Code,Walk Score,Bike Score,Population
count,2500.000000,2500.000000,2500.00000,2.500000e+03
mean,20787.882800,36.364800,43.45200,6.687973e+04
std,32514.371193,15.711144,11.58688,2.157376e+05
min,0.000000,2.000000,7.00000,1.652200e+04
25%,0.000000,25.000000,36.00000,2.246775e+04
50%,0.000000,35.000000,43.00000,3.262700e+04
75%,36675.750000,44.250000,50.00000,5.857000e+04
max,99336.000000,96.000000,96.00000,8.175133e+06


In [51]:
# Quartile cut the Walk Score data, to see how it is evenly distributed

pd.qcut(walk['Walk Score'], q=5)

0        (31.0, 38.0]
1        (23.0, 31.0]
2        (31.0, 38.0]
3        (23.0, 31.0]
4        (31.0, 38.0]
            ...      
2495     (38.0, 48.0]
2496     (31.0, 38.0]
2497    (1.999, 23.0]
2498     (23.0, 31.0]
2499     (23.0, 31.0]
Name: Walk Score, Length: 2500, dtype: category
Categories (5, interval[float64]): [(1.999, 23.0] < (23.0, 31.0] < (31.0, 38.0] < (38.0, 48.0] < (48.0, 96.0]]

In [52]:
# Bin the Walk Score data into categories and ranges

bins = [1.999, 23.0, 31.0, 38.0, 48.0, 96.0]
labels = ["Lowest Walk Score", "Lower Walk Score", "Average Walk Score", "Higher Walk Score", "Highest Walk Score"]
walk['Walk Score Categories'] = pd.cut(walk['Walk Score'], bins=bins, labels=labels)

bins = [1.999, 23.0, 31.0, 38.0, 48.0, 96.0]
labels = ["1.999-23.0", "23,0-31.0", "31.0-38.0", "38.0-48.0", "48.0-96.0"]
walk['Walk Score Ranges'] = pd.cut(walk['Walk Score'], bins=bins, labels=labels)

walk

,Zip Code,Walk Score,Transit Score,Bike Score,Population,Location,Walk Score Categories,Walk Score Ranges
0,35211,35,25,31,212237,"Birmingham, Alabama",Average Walk Score,31.0-38.0
1,36109,26,16,38,205764,"Montgomery, Alabama",Lower Walk Score,"23,0-31.0"
2,36605,32,--,39,195111,"Mobile, Alabama",Average Walk Score,31.0-38.0
3,35810,24,13,40,180105,"Huntsville, Alabama",Lower Walk Score,"23,0-31.0"
4,0,33,--,37,90468,"Tuscaloosa, Alabama",Average Walk Score,31.0-38.0
...,...,...,...,...,...,...,...,...
2495,0,39,--,44,55316,"Casper, Wyoming",Higher Walk Score,38.0-48.0
2496,0,34,--,67,30816,"Laramie, Wyoming",Average Walk Score,31.0-38.0
2497,0,21,--,34,29087,"Gillette, Wyoming",Lowest Walk Score,1.999-23.0
2498,0,29,--,32,23036,"Rock Springs, Wyoming",Lower Walk Score,"23,0-31.0"


In [53]:
# Quartile cut the Bike Score data, to see how it is evenly distributed

pd.qcut(walk['Bike Score'], q=5)

0       (6.999, 34.0]
1        (34.0, 41.0]
2        (34.0, 41.0]
3        (34.0, 41.0]
4        (34.0, 41.0]
            ...      
2495     (41.0, 46.0]
2496     (52.0, 96.0]
2497    (6.999, 34.0]
2498    (6.999, 34.0]
2499     (41.0, 46.0]
Name: Bike Score, Length: 2500, dtype: category
Categories (5, interval[float64]): [(6.999, 34.0] < (34.0, 41.0] < (41.0, 46.0] < (46.0, 52.0] < (52.0, 96.0]]

In [54]:
# Bin the Bike Score data into categories and ranges

bins = [6.999, 34.0, 41.0, 46.0, 52.0, 96.0]
labels = ["Lowest Bike Scores", "Lower Bike Scores", "Average Bike Scores", "Higher Bike Scores", "Highest Bike Scores"]
walk['Bike Score Categories'] = pd.cut(walk['Bike Score'], bins=bins, labels=labels)

bins = [6.999, 34.0, 41.0, 46.0, 52.0, 96.0]
labels = ["6.999-34.0", "34.0-41.0", "41.0-46.0", "46.0-52.0", "52.0-96.0"]
walk['Bike Score Ranges'] = pd.cut(walk['Bike Score'], bins=bins, labels=labels)

walk

,Zip Code,Walk Score,Transit Score,Bike Score,Population,Location,Walk Score Categories,Walk Score Ranges,Bike Score Categories,Bike Score Ranges
0,35211,35,25,31,212237,"Birmingham, Alabama",Average Walk Score,31.0-38.0,Lowest Bike Scores,6.999-34.0
1,36109,26,16,38,205764,"Montgomery, Alabama",Lower Walk Score,"23,0-31.0",Lower Bike Scores,34.0-41.0
2,36605,32,--,39,195111,"Mobile, Alabama",Average Walk Score,31.0-38.0,Lower Bike Scores,34.0-41.0
3,35810,24,13,40,180105,"Huntsville, Alabama",Lower Walk Score,"23,0-31.0",Lower Bike Scores,34.0-41.0
4,0,33,--,37,90468,"Tuscaloosa, Alabama",Average Walk Score,31.0-38.0,Lower Bike Scores,34.0-41.0
...,...,...,...,...,...,...,...,...,...,...
2495,0,39,--,44,55316,"Casper, Wyoming",Higher Walk Score,38.0-48.0,Average Bike Scores,41.0-46.0
2496,0,34,--,67,30816,"Laramie, Wyoming",Average Walk Score,31.0-38.0,Highest Bike Scores,52.0-96.0
2497,0,21,--,34,29087,"Gillette, Wyoming",Lowest Walk Score,1.999-23.0,Lowest Bike Scores,6.999-34.0
2498,0,29,--,32,23036,"Rock Springs, Wyoming",Lower Walk Score,"23,0-31.0",Lowest Bike Scores,6.999-34.0


In [55]:
# Replace "--" in Transit Score data with "0"

walk["Transit Score"] = walk['Transit Score'].str.replace('--', '0', regex=False)

In [56]:
# Make Transit Score data int

walk['Transit Score'] = walk['Transit Score'].astype('int')

In [57]:
walk.describe()

,Zip Code,Walk Score,Transit Score,Bike Score,Population
count,2500.000000,2500.000000,2500.000000,2500.00000,2.500000e+03
mean,20787.882800,36.364800,13.397600,43.45200,6.687973e+04
std,32514.371193,15.711144,17.595595,11.58688,2.157376e+05
min,0.000000,2.000000,0.000000,7.00000,1.652200e+04
25%,0.000000,25.000000,0.000000,36.00000,2.246775e+04
50%,0.000000,35.000000,0.000000,43.00000,3.262700e+04
75%,36675.750000,44.250000,29.000000,50.00000,5.857000e+04
max,99336.000000,96.000000,84.000000,96.00000,8.175133e+06


In [58]:
# Quartile cut the Bike Score data, to see how it is evenly distributed

pd.qcut(walk['Transit Score'], q=1)

# Leaving out Transit Score data for now, not enough data

0       (-0.001, 84.0]
1       (-0.001, 84.0]
2       (-0.001, 84.0]
3       (-0.001, 84.0]
4       (-0.001, 84.0]
             ...      
2495    (-0.001, 84.0]
2496    (-0.001, 84.0]
2497    (-0.001, 84.0]
2498    (-0.001, 84.0]
2499    (-0.001, 84.0]
Name: Transit Score, Length: 2500, dtype: category
Categories (1, interval[float64]): [(-0.001, 84.0]]

In [59]:
# Drop Zip Code because too granular, Population because it is from previous years (search for better walk scores), and drop Transit Score because too many missing values

walk = walk.drop(columns=['Zip Code', 'Population', 'Transit Score'])

In [60]:
# Check out cost data

cost

,City,State,Cost of Living Index
0,Aberdeen,WA,95.5
1,Abilene,TX,89.6
2,Adrian,MI,89.9
3,Akron,OH,90.8
4,Alamogordo,NM,86.5
...,...,...,...
504,Vero Beach,FL,100.0
505,Weirton,WV,82.6
506,Wheeling,WV,84.5
507,New London,CT,107.3


In [61]:
# Replace state abbreviations with full state names

cost["State"] = cost['State'].str.replace('AK', 'Alaska', regex=False)
cost["State"] = cost['State'].str.replace('AL', 'Alabama', regex=False)
cost["State"] = cost['State'].str.replace('AR', 'Arkansas', regex=False)
cost["State"] = cost['State'].str.replace('AS', 'American Samoa', regex=False)
cost["State"] = cost['State'].str.replace('AZ', 'Arizona', regex=False)
cost["State"] = cost['State'].str.replace('CA', 'California', regex=False)
cost["State"] = cost['State'].str.replace('CO', 'Colorado', regex=False)
cost["State"] = cost['State'].str.replace('CT', 'Connecticut', regex=False)
cost["State"] = cost['State'].str.replace('DC', 'District of Columbia', regex=False)
cost["State"] = cost['State'].str.replace('DE', 'Delaware', regex=False)
cost["State"] = cost['State'].str.replace('FL', 'Florida', regex=False)
cost["State"] = cost['State'].str.replace('GA', 'Georgia', regex=False)
cost["State"] = cost['State'].str.replace('GU', 'Guam', regex=False)
cost["State"] = cost['State'].str.replace('HI', 'Hawaii', regex=False)
cost["State"] = cost['State'].str.replace('IA', 'Iowa', regex=False)
cost["State"] = cost['State'].str.replace('ID', 'Idaho', regex=False)
cost["State"] = cost['State'].str.replace('IL', 'Illinois', regex=False)
cost["State"] = cost['State'].str.replace('IN', 'Indiana', regex=False)
cost["State"] = cost['State'].str.replace('KS', 'Kansas', regex=False)
cost["State"] = cost['State'].str.replace('KY', 'Kentucky', regex=False)
cost["State"] = cost['State'].str.replace('LA', 'Louisiana', regex=False)
cost["State"] = cost['State'].str.replace('MA', 'Massachusetts', regex=False)
cost["State"] = cost['State'].str.replace('MD', 'Maryland', regex=False)
cost["State"] = cost['State'].str.replace('ME', 'Maine', regex=False)
cost["State"] = cost['State'].str.replace('MI', 'Michigan', regex=False)
cost["State"] = cost['State'].str.replace('MN', 'Minnesota', regex=False)
cost["State"] = cost['State'].str.replace('MO', 'Missouri', regex=False)
cost["State"] = cost['State'].str.replace('MP', 'Northern Mariana Islands', regex=False)
cost["State"] = cost['State'].str.replace('MS', 'Mississippi', regex=False)
cost["State"] = cost['State'].str.replace('MT', 'Montana', regex=False)
cost["State"] = cost['State'].str.replace('NA', 'National', regex=False)
cost["State"] = cost['State'].str.replace('NC', 'North Carolina', regex=False)
cost["State"] = cost['State'].str.replace('ND', 'North Dakota', regex=False)
cost["State"] = cost['State'].str.replace('NE', 'Nebraska', regex=False)
cost["State"] = cost['State'].str.replace('NH', 'New Hampshire', regex=False)
cost["State"] = cost['State'].str.replace('NJ', 'New Jersey', regex=False)
cost["State"] = cost['State'].str.replace('NM', 'New Mexico', regex=False)
cost["State"] = cost['State'].str.replace('NV', 'Nevada', regex=False)
cost["State"] = cost['State'].str.replace('NY', 'New York', regex=False)
cost["State"] = cost['State'].str.replace('OH', 'Ohio', regex=False)
cost["State"] = cost['State'].str.replace('OK', 'Oklahoma', regex=False)
cost["State"] = cost['State'].str.replace('OR', 'Oregon', regex=False)
cost["State"] = cost['State'].str.replace('PA', 'Pennsylvania', regex=False)
cost["State"] = cost['State'].str.replace('PR', 'Puerto Rico', regex=False)
cost["State"] = cost['State'].str.replace('RI', 'Rhode Island', regex=False)
cost["State"] = cost['State'].str.replace('SC', 'South Carolina', regex=False)
cost["State"] = cost['State'].str.replace('SD', 'South Dakota', regex=False)
cost["State"] = cost['State'].str.replace('TN', 'Tennessee', regex=False)
cost["State"] = cost['State'].str.replace('TX', 'Texas', regex=False)
cost["State"] = cost['State'].str.replace('UT', 'Utah', regex=False)
cost["State"] = cost['State'].str.replace('VA', 'Virginia', regex=False)
cost["State"] = cost['State'].str.replace('VI', 'Virgin Islands', regex=False)
cost["State"] = cost['State'].str.replace('VT', 'Vermont', regex=False)
cost["State"] = cost['State'].str.replace('WA', 'Washington', regex=False)
cost["State"] = cost['State'].str.replace('WI', 'Wisconsin', regex=False)
cost["State"] = cost['State'].str.replace('WV', 'West Virginia', regex=False)
cost["State"] = cost['State'].str.replace('WY', 'Wyoming', regex=False)

In [62]:
# Make new Location column with City and State data, so the data can be joined with the other data on this column

cost['Location'] = cost['City'] + ',' + ' ' + cost['State']
cost

,City,State,Cost of Living Index,Location
0,Aberdeen,Washington,95.5,"Aberdeen, Washington"
1,Abilene,Texas,89.6,"Abilene, Texas"
2,Adrian,Michigan,89.9,"Adrian, Michigan"
3,Akron,Ohio,90.8,"Akron, Ohio"
4,Alamogordo,New Mexico,86.5,"Alamogordo, New Mexico"
...,...,...,...,...
504,Vero Beach,Florida,100.0,"Vero Beach, Florida"
505,Weirton,West Virginia,82.6,"Weirton, West Virginia"
506,Wheeling,West Virginia,84.5,"Wheeling, West Virginia"
507,New London,Connecticut,107.3,"New London, Connecticut"


In [63]:
# Drop City and State columns since they are redundant with Location and they already exist in first merged data

cost = cost.drop(columns=['City', 'State'])

In [64]:
# Bin the cost data according to quartiles

# Describe the data to begin binning data with pandas cut or qcut

cost.describe()

,Cost of Living Index
count,509.000000
mean,97.500196
std,13.915860
min,78.800000
25%,89.000000
50%,93.100000
75%,101.900000
max,183.000000


In [65]:
# Quartile cut the Walk Score data, to see how it is evenly distributed

pd.qcut(cost['Cost of Living Index'], q=5)

0                  (91.4, 96.18]
1                   (88.5, 91.4]
2                   (88.5, 91.4]
3                   (88.5, 91.4]
4      (78.79899999999999, 88.5]
                 ...            
504              (96.18, 103.74]
505    (78.79899999999999, 88.5]
506    (78.79899999999999, 88.5]
507              (103.74, 183.0]
508                 (88.5, 91.4]
Name: Cost of Living Index, Length: 509, dtype: category
Categories (5, interval[float64]): [(78.79899999999999, 88.5] < (88.5, 91.4] < (91.4, 96.18] < (96.18, 103.74] < (103.74, 183.0]]

In [66]:
# Bin the Cost of Living Index data into categories and ranges

bins = [78.79899999999999, 88.5, 91.4, 96.18, 103.74, 183.0]
labels = ["Lowest Cost of Living Index", "Lower Cost of Living Index", "Average Cost of Living Index", "Higher Cost of Living Index", "Highest Cost of Living Index"]
cost['Cost of Living Index Categories'] = pd.cut(cost['Cost of Living Index'], bins=bins, labels=labels)

bins = [78.79899999999999, 88.5, 91.4, 96.18, 103.74, 183.0]
labels = ["78.79-88.5", "88.5-91.4", "91.4-96.18", "96.18-103.74", "103.74-183.0"]
cost['Cost of Living Index Ranges'] = pd.cut(cost['Cost of Living Index'], bins=bins, labels=labels)

cost

,Cost of Living Index,Location,Cost of Living Index Categories,Cost of Living Index Ranges
0,95.5,"Aberdeen, Washington",Average Cost of Living Index,91.4-96.18
1,89.6,"Abilene, Texas",Lower Cost of Living Index,88.5-91.4
2,89.9,"Adrian, Michigan",Lower Cost of Living Index,88.5-91.4
3,90.8,"Akron, Ohio",Lower Cost of Living Index,88.5-91.4
4,86.5,"Alamogordo, New Mexico",Lowest Cost of Living Index,78.79-88.5
...,...,...,...,...
504,100.0,"Vero Beach, Florida",Higher Cost of Living Index,96.18-103.74
505,82.6,"Weirton, West Virginia",Lowest Cost of Living Index,78.79-88.5
506,84.5,"Wheeling, West Virginia",Lowest Cost of Living Index,78.79-88.5
507,107.3,"New London, Connecticut",Highest Cost of Living Index,103.74-183.0


In [67]:
# Check shape of merged_population_rent_crime data

merged_population_rent_crime.shape

(1626, 24)

In [68]:
# Merge the walk data with the merged_population_rent_crime data

merged_population_rent_crime_walk = pd.merge(left=merged_population_rent_crime, right=walk, left_on='Location', right_on='Location')
merged_population_rent_crime_walk

# When is this walk_data from? Consider dropping the populatin data from walk_data since we already have 2019 population data, and 2019 crime population
# Also consider dropping Zip Code from the walk data because it is too high granularity and it does not match the true area of different areas with different rental rates

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,...,Rental Rate Categories,Rental Rate Ranges,Crime Rate Categories,Crime Rate Ranges,Walk Score,Bike Score,Walk Score Categories,Walk Score Ranges,Bike Score Categories,Bike Score Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,...,Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,...,Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,...,Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,...,Average Rent,"$1,294.50-1,549",Average Crime,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,...,Higher Rent,"$1,549-1,891.50",Average Crime,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,...,Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0
1622,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,...,Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0
1623,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,...,Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0
1624,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,...,Low Rent,"$581.99-1,294.50",Lower Crime,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0


In [69]:
# Merge the cost data with the merged_population_rent_crime_walk data

merged_population_rent_crime_walk_cost = pd.merge(left=merged_population_rent_crime_walk, right=cost, left_on='Location', right_on='Location')
merged_population_rent_crime_walk_cost

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,...,Crime Rate Ranges,Walk Score,Bike Score,Walk Score Categories,Walk Score Ranges,Bike Score Categories,Bike Score Ranges,Cost of Living Index,Cost of Living Index Categories,Cost of Living Index Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,...,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,...,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,...,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,...,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,...,"7,951.131-9,683.737",41,56,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,...,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18
1622,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,...,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18
1623,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,...,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18
1624,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,...,"5,831.013-7,951.131",63,59,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18


In [70]:
# Count unique Locations to be sure we are not losing any data with our merges, as we should not

merged_population_rent_crime['Location'].nunique()

68

In [71]:
merged_population_rent_crime_walk['Location'].nunique()

68

In [72]:
merged_population_rent_crime_walk_cost['Location'].nunique()

68

In [73]:
# Calculating Livability Score with Walk Score, Bike Score, Cost of Living Index, and Crime Rate (Subtracted from the max crime rate of 16274.419781 to reverse the scores)

merged_population_rent_crime_walk_cost['Livability Score'] = merged_population_rent_crime_walk_cost["Walk Score"] + merged_population_rent_crime_walk_cost["Bike Score"] + merged_population_rent_crime_walk_cost["Cost of Living Index"] + 16274.419781 - merged_population_rent_crime_walk_cost["Crime Rate"]

# https://scikit-learn.org/stable/modules/preprocessing.html
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler

In [74]:
# Scale the Livability score then multiply it by 100 to make the scores between 0 and 100

In [75]:
# Bin the cost data according to quartiles

# Describe the data to begin binning data with pandas cut or qcut

merged_population_rent_crime_walk_cost.describe()

,2019 Population,2019 Rental Rates,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Rate,Walk Score,Bike Score,Cost of Living Index,Livability Score
count,1.626000e+03,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000,1626.000000
mean,8.890544e+05,1674.565806,7292.163592,111.785363,549.460025,2175.714022,4455.204182,31785.571341,4757.443419,22929.142681,4098.985240,170.505535,8782.427187,52.062116,57.291513,111.340959,7712.687182
std,7.568980e+05,607.902951,7658.789061,139.237847,516.120724,2582.431727,4592.876884,26879.392882,4074.197256,19999.580116,3342.446938,170.794557,3034.576864,16.213550,11.676507,22.004236,3028.159189
min,4.927100e+04,582.000000,135.000000,0.000000,18.000000,13.000000,77.000000,1069.000000,163.000000,728.000000,81.000000,1.000000,1972.791138,13.000000,30.000000,88.000000,218.000000
25%,3.313600e+05,1294.500000,1686.000000,28.000000,127.000000,409.500000,1186.000000,9790.000000,1302.000000,6997.000000,1274.000000,38.000000,6949.239498,41.000000,49.000000,100.300000,5501.802351
50%,6.547410e+05,1549.000000,4559.000000,40.000000,358.000000,1294.000000,2517.000000,27141.000000,4114.000000,16395.000000,3645.000000,108.000000,8162.208385,49.000000,56.000000,102.900000,8293.511396
75%,1.021795e+06,1891.500000,11101.000000,131.000000,713.000000,3055.000000,7336.000000,48780.000000,7210.000000,39427.000000,6432.000000,201.000000,10968.517430,65.000000,68.000000,116.800000,9532.980283
max,2.693976e+06,6373.000000,25532.000000,492.000000,1761.000000,9147.000000,15296.000000,101750.000000,17038.000000,71614.000000,13098.000000,789.000000,16274.419781,87.000000,86.000000,183.000000,14454.628643


In [76]:
# Quartile cut the Livability Score data, to see how it is evenly distributed

pd.qcut(merged_population_rent_crime_walk_cost['Livability Score'], q=5)

0        (6756.682, 8531.989]
1        (6756.682, 8531.989]
2        (6756.682, 8531.989]
3        (6756.682, 8531.989]
4        (6756.682, 8531.989]
                ...          
1621    (8531.989, 10735.507]
1622    (8531.989, 10735.507]
1623    (8531.989, 10735.507]
1624    (8531.989, 10735.507]
1625    (8531.989, 10735.507]
Name: Livability Score, Length: 1626, dtype: category
Categories (5, interval[float64]): [(217.999, 5161.602] < (5161.602, 6756.682] < (6756.682, 8531.989] < (8531.989, 10735.507] < (10735.507, 14454.629]]

In [77]:
# Bin the Cost of Living Index data into categories and ranges

bins = [217.999, 5161.602, 6756.682, 8531.989, 10735.507, 14454.629]
labels = ["Lowest Livability Scores", "Lower Livability Scores", "Average Livability Scores", "Higher Livability Scores", "Highest Livability Scores"]
merged_population_rent_crime_walk_cost['Livability Score Categories'] = pd.cut(merged_population_rent_crime_walk_cost['Livability Score'], bins=bins, labels=labels)

bins = [217.999, 5161.602, 6756.682, 8531.989, 10735.507, 14454.629]
labels = ["217.99-5,161.60", "5,161.60-6,756.68", "6,756.68-8,531.98", "8,531.98-10,735.50", "10,735.50-14,454.62"]
merged_population_rent_crime_walk_cost['Livability Score Ranges'] = pd.cut(merged_population_rent_crime_walk_cost['Livability Score'], bins=bins, labels=labels)

merged_population_rent_crime_walk_cost

,Location,2019 Population,Town or City,2019 Rental Rates,State,City,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,...,Walk Score Categories,Walk Score Ranges,Bike Score Categories,Bike Score Ranges,Cost of Living Index,Cost of Living Index Categories,Cost of Living Index Ranges,Livability Score,Livability Score Categories,Livability Score Ranges
0,"Phoenix, Arizona",1680992,city,1424,Arizona,Phoenix,11803,131,1139,3197,...,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0,8401.333638,Average Livability Scores,"6,756.68-8,531.98"
1,"Phoenix, Arizona",1680992,city,1522,Arizona,Phoenix,11803,131,1139,3197,...,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0,8401.333638,Average Livability Scores,"6,756.68-8,531.98"
2,"Phoenix, Arizona",1680992,city,1461,Arizona,Phoenix,11803,131,1139,3197,...,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0,8401.333638,Average Livability Scores,"6,756.68-8,531.98"
3,"Phoenix, Arizona",1680992,city,1549,Arizona,Phoenix,11803,131,1139,3197,...,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0,8401.333638,Average Livability Scores,"6,756.68-8,531.98"
4,"Phoenix, Arizona",1680992,city,1611,Arizona,Phoenix,11803,131,1139,3197,...,Higher Walk Score,38.0-48.0,Highest Bike Scores,52.0-96.0,105.8,Highest Cost of Living Index,103.74-183.0,8401.333638,Average Livability Scores,"6,756.68-8,531.98"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,"Milwaukee, Wisconsin",590157,city,839,Wisconsin,Milwaukee,7874,97,427,1911,...,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18,8672.961250,Higher Livability Scores,"8,531.98-10,735.50"
1622,"Milwaukee, Wisconsin",590157,city,948,Wisconsin,Milwaukee,7874,97,427,1911,...,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18,8672.961250,Higher Livability Scores,"8,531.98-10,735.50"
1623,"Milwaukee, Wisconsin",590157,city,973,Wisconsin,Milwaukee,7874,97,427,1911,...,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18,8672.961250,Higher Livability Scores,"8,531.98-10,735.50"
1624,"Milwaukee, Wisconsin",590157,city,792,Wisconsin,Milwaukee,7874,97,427,1911,...,Highest Walk Score,48.0-96.0,Highest Bike Scores,52.0-96.0,94.8,Average Cost of Living Index,91.4-96.18,8672.961250,Higher Livability Scores,"8,531.98-10,735.50"


In [78]:
# Calculate the mean values of duplication Location entries

merged_population_rent_crime_walk_cost = merged_population_rent_crime_walk_cost.groupby('Location').mean().reset_index()
merged_population_rent_crime_walk_cost

,Location,2019 Population,2019 Rental Rates,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,Arson,Crime Rate,Walk Score,Bike Score,Cost of Living Index,Livability Score
0,"Akron, Ohio",197597.0,678.000000,1782.0,27.0,181.0,328.0,1246.0,6568.0,1686.0,4305.0,577.0,65.0,8484.440553,46,46,90.8,7972.779228
1,"Albany, New York",96460.0,1659.000000,736.0,4.0,60.0,189.0,483.0,2919.0,445.0,2315.0,159.0,16.0,7594.857972,65,56,101.6,8902.161809
2,"Allentown, Pennsylvania",121442.0,1373.333333,471.0,7.0,64.0,170.0,230.0,2782.0,521.0,2018.0,243.0,5.0,5361.407091,70,44,100.2,11127.212690
3,"Austin, Texas",978908.0,1598.500000,3953.0,32.0,534.0,971.0,2416.0,36588.0,4344.0,29216.0,3028.0,103.0,8293.424918,41,54,107.7,8183.694863
4,"Bakersfield, California",384145.0,1340.000000,1766.0,34.0,116.0,701.0,915.0,16074.0,3888.0,9277.0,2909.0,470.0,9410.509053,34,44,103.6,7045.510728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,"Tucson, Arizona",548073.0,1248.714286,3775.0,40.0,527.0,1105.0,2103.0,17943.0,2497.0,13196.0,2250.0,142.0,7951.130598,42,67,99.7,8531.989183
64,"Tulsa, Oklahoma",401190.0,1206.000000,3964.0,55.0,341.0,718.0,2850.0,21336.0,4846.0,13457.0,3033.0,209.0,12664.572896,39,47,88.7,3784.546885
65,"Virginia Beach, Virginia",449974.0,1356.707317,581.0,30.0,79.0,196.0,276.0,7906.0,530.0,6797.0,579.0,34.0,3779.773942,32,44,101.1,12671.745839
66,"Wichita, Kansas",389938.0,857.000000,4451.0,35.0,367.0,461.0,3588.0,20759.0,2677.0,15777.0,2305.0,125.0,12962.317086,35,48,88.0,3483.102695


In [79]:
# Export merged_population_rent_crime_walk_cost with binned categories for csv and continue modeling in another notebook

merged_population_rent_crime_walk_cost.to_csv('../data/pop_rent_crime_walk_cost_livability_bins.csv')

In [80]:
# Consider scaling data between 0-100, like Livability Score since we calculated it, perhaps then it is more interpretable by users.